# Semantic search quick start

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/elastic/elasticsearch-labs/blob/main/notebooks/search/00-quick-start.ipynb)

This interactive notebook will introduce you to some basic operations with Elasticsearch, using the official [Elasticsearch Python client](https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/connecting.html).
You'll perform semantic search using [Sentence Transformers](https://www.sbert.net) for text embedding. Learn how to integrate traditional text-based search with semantic search, for a hybrid search system.

## Create Elastic Cloud deployment

If you don't have an Elastic Cloud deployment, sign up [here](https://cloud.elastic.co/registration?onboarding_token=vectorsearch&utm_source=github&utm_content=elasticsearch-labs-notebook) for a free trial.

Once logged in to your Elastic Cloud account, go to the [Create deployment](https://cloud.elastic.co/deployments/create) page and select **Create deployment**. Leave all settings with their default values.

## Install packages and import modules

To get started, we'll need to connect to our Elastic deployment using the Python client.
Because we're using an Elastic Cloud deployment, we'll use the **Cloud ID** to identify our deployment.

First we need to install the `elasticsearch` Python client.

In [ ]:
!pip install -qU "elasticsearch<9" sentence-transformers==2.7.0

# Setup the Embedding Model

For this example, we're using `all-MiniLM-L6-v2`, part of the `sentence_transformers` library. You can read more about this model on [Huggingface](https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2).

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")

## Initialize the Elasticsearch client

Now we can instantiate the [Elasticsearch python client](https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/index.html), providing the cloud id and password in your deployment.

In [ ]:
from elasticsearch import Elasticsearch
from getpass import getpass

# https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud#finding-your-cloud-id
ELASTIC_CLOUD_ID = getpass("Elastic Cloud ID: ")

# https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud#creating-an-api-key
ELASTIC_API_KEY = getpass("Elastic Api Key: ")

# Create the client instance
client = Elasticsearch(
    # For local development
    # hosts=["http://localhost:9200"]
    cloud_id=ELASTIC_CLOUD_ID,
    api_key=ELASTIC_API_KEY,
)

If you're running Elasticsearch locally or self-managed, you can pass in the Elasticsearch host instead. [Read more](https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/connecting.html#_verifying_https_with_certificate_fingerprints_python_3_10_or_later) on how to connect to Elasticsearch locally.

### Enable Telemetry

Knowing that you are using this notebook helps us decide where to invest our efforts to improve our products. We would like to ask you that you run the following code to let us gather anonymous usage statistics. See [telemetry.py](https://github.com/elastic/elasticsearch-labs/blob/main/telemetry/telemetry.py) for details. Thank you!

In [ ]:
!curl -O -s https://raw.githubusercontent.com/elastic/elasticsearch-labs/main/telemetry/telemetry.py
from telemetry import enable_telemetry

client = enable_telemetry(client, "00-quick-start")

### Test the Client
Before you continue, confirm that the client has connected with this test.

In [ ]:
print(client.info())

## Index some test data

Our client is set up and connected to our Elastic deployment.
Now we need some data to test out the basics of Elasticsearch queries.
We'll use a small index of books with the following fields:

- `title`
- `authors`
- `publish_date`
- `num_reviews`
- `publisher`

### Create an index

First ensure that you do not have a previously created index with the name `book_index`.

In [ ]:
client.indices.delete(index="book_index", ignore_unavailable=True)

🔐 NOTE: at any time you can come back to this section and run the `delete` function above to remove your index and start from scratch.

Let's create an Elasticsearch index with the correct mappings for our test data.

In [ ]:
# Define the mapping
mappings = {
    "properties": {
        "title_vector": {
            "type": "dense_vector",
            "dims": 384,
            "index": "true",
            "similarity": "cosine",
        }
    }
}

# Create the index
client.indices.create(index="book_index", mappings=mappings)

### Index test data

Run the following command to upload some test data, containing information about 10 popular programming books from this [dataset](https://raw.githubusercontent.com/elastic/elasticsearch-labs/main/notebooks/search/data.json).
`model.encode` will encode the text into a vector on the fly, using the model we initialized earlier.

In [ ]:
import json
from urllib.request import urlopen

url = "https://raw.githubusercontent.com/elastic/elasticsearch-labs/main/notebooks/search/data.json"
response = urlopen(url)
books = json.loads(response.read())

operations = []
for book in books:
    operations.append({"index": {"_index": "book_index"}})
    # Transforming the title into an embedding using the model
    book["title_vector"] = model.encode(book["title"]).tolist()
    operations.append(book)
client.bulk(index="book_index", operations=operations, refresh=True)

## Aside: Pretty printing Elasticsearch responses

Your API calls will return hard-to-read nested JSON.
We'll create a little function called `pretty_response` to return nice, human-readable outputs from our examples.

In [ ]:
def pretty_response(response):
    if len(response["hits"]["hits"]) == 0:
        print("Your search returned no results.")
    else:
        for hit in response["hits"]["hits"]:
            id = hit["_id"]
            publication_date = hit["_source"]["publish_date"]
            score = hit["_score"]
            title = hit["_source"]["title"]
            summary = hit["_source"]["summary"]
            publisher = hit["_source"]["publisher"]
            num_reviews = hit["_source"]["num_reviews"]
            authors = hit["_source"]["authors"]
            pretty_output = f"\nID: {id}\nPublication date: {publication_date}\nTitle: {title}\nSummary: {summary}\nPublisher: {publisher}\nReviews: {num_reviews}\nAuthors: {authors}\nScore: {score}"
            print(pretty_output)

## Making queries

Now that we have indexed the books, we want to perform a semantic search for books that are similar to a given query.
We embed the query and perform a search.

In [ ]:
response = client.search(
    index="book_index",
    knn={
        "field": "title_vector",
        "query_vector": model.encode("clear"),
        "k": 3,
        "num_candidates": 3,
    },
)

pretty_response(response)

## Filtering

Filter context is mostly used for filtering structured data. For example, use filter context to answer questions like:

- _Does this timestamp fall into the range 2015 to 2016?_
- _Is the status field set to "published"?_

Filter context is in effect whenever a query clause is passed to a filter parameter, such as the `filter` or `must_not` parameters in a `bool` query.

[Learn more](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-filter-context.html#filter-context) about filter context in the Elasticsearch docs.

### Example: Keyword Filtering

This is an example of adding a keyword filter to the query.

The example retrieves the top books that are similar to "javascript books" based on their title vectors, and also Addison-Wesley as publisher.

In [ ]:
response = client.search(
    index="book_index",
    knn={
        "field": "title_vector",
        "query_vector": model.encode("javascript books"),
        "k": 10,
        "num_candidates": 100,
        "filter": {"term": {"publisher.keyword": "addison-wesley"}},
    },
)

pretty_response(response)

In [ ]:
from datetime import datetime, timedelta
# Calcula la fecha de hace dos años desde hoy
two_years_ago = (datetime.now() - timedelta(days=8*365)).strftime('%Y-%m-%d')

response = client.search(
    index="book_index",
    knn={
        "field": "title_vector",
        "query_vector": model.encode("javascript books"), # o la consulta que desees
        "k": 10,
        "num_candidates": 100,
        "filter": {
            "range": {
                "publish_date": {
                    "gte": two_years_ago,
                    "lte": "now" # Puedes usar "now" para la fecha actual
                }
            }
        },
    },
)

# Asume que tienes una función pretty_response definida
# Ejemplo:
# import json
# def pretty_response(response):
#     print(json.dumps(response.body, indent=2, ensure_ascii=False))

pretty_response(response)

1. Buscar Libros por Autor ✍️
Puedes buscar utilizando el campo authors (analizado, para búsquedas flexibles) o authors.keyword (no analizado, para búsquedas exactas).

Búsqueda flexible (usando authors de tipo text)
Esto encontrará libros donde "Gabriel García" aparezca en el campo de autores, incluso si hay otros autores o si el nombre tiene variaciones que el analizador pueda manejar.

Python

In [ ]:
def pretty_print(response):
    """Función auxiliar para imprimir respuestas JSON de forma legible."""
    print(json.dumps(response, indent=2, ensure_ascii=False))

# Nombre del índice
INDEX_NAME = "book_index"

print("## 1.1 Búsqueda flexible de autor:")
response = client.search(
    index=INDEX_NAME,
    query={
        "match": {
            "authors": "ericc mathes"
        }
    },
    size=5 # Limitar a 5 resultados
)
pretty_response(response.body)

In [ ]:
print("\n## Búsqueda flexible de autor (incluyendo solo 'title', 'authors', 'publish_date'):")
response_specific_fields = client.search(
    index=INDEX_NAME,
    query={
        "match": {
            "authors": "eric"
        }
    },
    size=1,
    _source=["title", "authors", "publish_date"] # <--- AQUÍ SE ESPECIFICAN LOS CAMPOS A INCLUIR
)
pretty_print(response_specific_fields.body)

In [ ]:
print("## Búsqueda flexible de autor (excluyendo 'title_vector'):")
response = client.search(
    index=INDEX_NAME,
    query={
        "match": {
            "authors": "eric"
        }
    },
    size=1, # Limitar a 1 resultado para el ejemplo
    _source_excludes=["title_vector"] # <--- AQUÍ SE EXCLUYE EL CAMPO
)
pretty_print(response.body)

In [ ]:
print("## Búsqueda por Prefijo (ej: 'eri'):")
query_term = "eric"
response = client.search(
    index=INDEX_NAME,
    query={
        "match_phrase_prefix": {
            "authors": {
                "query": query_term
            }
        }
    },
    size=5,
    _source_excludes=["title_vector"]
)
pretty_print(response.body)
# Debería encontrar 'eric matthes'

In [ ]:
print("\n## Búsqueda con Fuzziness (ej: 'erric'):")
query_term = "rric"
response = client.search(
    index=INDEX_NAME,
    query={
        "match": {
            "authors": {
                "query": query_term,
                "fuzziness": "AUTO" # O un valor como 1 o 2 (distancia de Levenshtein)
                                # "AUTO" genera distancias de edición basadas en la longitud del término.
            }
        }
    },
    size=5,
    _source_excludes=["title_vector"]
)
pretty_print(response.body)
# Debería encontrar 'eric matthes' si la fuzziness es adecuada.

In [ ]:
print("\n## Búsqueda con Comodines usando query_string:")

# Ejemplo para "eri" (como prefijo)
query_wildcard_prefix = "eri*"
print(f"--- Buscando con: {query_wildcard_prefix} ---")
response_prefix = client.search(
    index=INDEX_NAME,
    query={
        "query_string": {
            "query": query_wildcard_prefix,
            "default_field": "authors"
        }
    },
    size=5,
     _source_excludes=["title_vector"]
)
pretty_print(response_prefix.body)



In [ ]:
# Ejemplo para "ric" (como parte del término)
query_wildcard_contains = "*ric*" # Podría ser "*ric" si solo buscas al final
print(f"\n--- Buscando con: {query_wildcard_contains} ---")
response_contains = client.search(
    index=INDEX_NAME,
    query={
        "query_string": {
            "query": query_wildcard_contains,
            "default_field": "authors"
        }
    },
    size=5,
     _source_excludes=["title_vector"]
)
pretty_print(response_contains.body)
# Esta consulta ('*ric*') debería encontrar 'eric matthes'.

In [ ]:
print("\n## Búsqueda con simple_query_string:")

# Para prefijo "eri"
query_sqs_prefix = "eri*"
print(f"--- Buscando con simple_query_string (prefijo): {query_sqs_prefix} ---")
response_sqs_prefix = client.search(
    index=INDEX_NAME,
    query={
        "simple_query_string": {
            "query": query_sqs_prefix,
            "fields": ["authors"], # Especifica los campos donde buscar
            "default_operator": "AND"
        }
    },
    size=5,
     _source_excludes=["title_vector"]
)
pretty_print(response_sqs_prefix.body)


# Para typo "erric" (usando fuzziness)
query_sqs_fuzzy = "erric~1" # El ~1 indica una distancia de edición de 1
print(f"\n--- Buscando con simple_query_string (fuzziness): {query_sqs_fuzzy} ---")
response_sqs_fuzzy = client.search(
    index=INDEX_NAME,
    query={
        "simple_query_string": {
            "query": query_sqs_fuzzy,
            "fields": ["authors"],
            "default_operator": "AND"
        }
    },
    size=5,
     _source_excludes=["title_vector"]
)
pretty_print(response_sqs_fuzzy.body)


# Para "ric" como parte de "eric"
query_sqs_contains = "*ric*"
print(f"\n--- Buscando con simple_query_string (contiene): {query_sqs_contains} ---")
response_sqs_contains = client.search(
    index=INDEX_NAME,
    query={
        "simple_query_string": {
            "query": query_sqs_contains,
            "fields": ["authors"],
            "default_operator": "AND"
        }
    },
    size=5,
     _source_excludes=["title_vector"]
)
pretty_print(response_sqs_contains.body)

In [ ]:
print("\n## Probando con query_string para '*ric*':")
query_term = "*ric*"
response_qs = client.search(
    index=INDEX_NAME,
    query={
        "query_string": {
            "query": query_term,
            "fields": ["authors"], # Puedes usar 'fields' o 'default_field'
            # "analyze_wildcard": False # Es el valor por defecto, pero puedes ser explícito
        }
    },
    size=5,
     _source_excludes=["title_vector"]
)
pretty_print(response_qs.body)

In [ ]:
print("\n## Probando simple_query_string simplificada para '*ric*':")
query_term = "*ric*"
response_sqs_simple = client.search(
    index=INDEX_NAME,
    query={
        "simple_query_string": {
            "query": query_term,
            "fields": ["authors"]
        }
    },
    size=5,
     _source_excludes=["title_vector"]
)
pretty_print(response_sqs_simple.body)

In [ ]:
GET book_index/_termvectors/_NVoQJcBYS182ubtUI05
{
  "fields": ["authors"]
}

In [ ]:
print("\n## 1.2 Búsqueda exacta de autor:")
response = client.search(
    index=INDEX_NAME,
    query={
        "term": {
            "authors.keyword": "david"
        }
    },
    size=5
)
pretty_response(response.body)

In [ ]:
print("\n## 2. Agregación de autores únicos:")
response = client.search(
    index=INDEX_NAME,
    size=0,  # No necesitamos los documentos, solo la agregación
    aggs={
        "unique_authors": {
            "terms": {
                "field": "authors.keyword",
                "size": 20  # Número de autores únicos a mostrar
            }
        }
    }
)
pretty_print(response.body['aggregations'])

In [ ]:
print("\n## 3. Destacar términos en autores:")
response = client.search(
    index=INDEX_NAME,
    query={
        "match": {
            "authors": "Martin" # Buscar libros que tengan "Martin" en los autores
        }
    },
    highlight={
        "fields": {
            "authors": {} # Configuración de resaltado por defecto para el campo 'authors'
        }
    },
    size=3,
    _source_excludes=["title_vector"]
)
pretty_print(response.body)

In [ ]:
print("\n## 4. Libros con más de un autor:")
response = client.search(
    index=INDEX_NAME,
    query={
        "bool": {
            "filter": {
                "script": {
                    "script": {
                        "source": "doc['authors.keyword'].size() > params.count",
                        "params": {
                            "count": 1
                        }
                    }
                }
            }
        }
    },
    size=5,
    _source=["title", "authors"] # Solo traer título y autores
)
pretty_print(response.body)

In [ ]:
print("\n## 5. Libros con más de dos autores:")
response = client.search(
    index=INDEX_NAME,
    query={
        "bool": {
            "filter": {
                "script": {
                    "script": {
                        "source": "doc['authors.keyword'].size() > params.count",
                        "params": {
                            "count": 2
                        }
                    }
                }
            }
        }
    },
    size=5,
    _source=["title", "authors"]
)
pretty_print(response.body)

In [ ]:
print("\n## 6. Facet por cantidad de autores:")
response = client.search(
    index=INDEX_NAME,
    size=0, # No necesitamos los documentos, solo la agregación
    aggs={
        "authors_count_facet": {
            "terms": {
                "script": {
                    "source": "doc['authors.keyword'].size()", # Devuelve el número de autores
                    "lang": "painless"
                },
                "size": 10 # Muestra los 10 conteos de autores más comunes
            }
        }
    }
)
pretty_print(response.body['aggregations'])

In [ ]:
from elasticsearch import Elasticsearch

# Asume que 'client' ya está inicializado
# client = Elasticsearch(...)
# INDEX_NAME = "book_index" # O un nuevo nombre como "book_index_ngram"

# 1. Definir settings y mappings
index_settings = {
    "analysis": {
        "analyzer": {
            "custom_trigram_analyzer": {
                "tokenizer": "standard",
                "filter": [
                    "lowercase",
                    "trigram_token_filter"
                ]
            }
        },
        "filter": {
            "trigram_token_filter": {
                "type": "ngram",
                "min_gram": 3,
                "max_gram": 3
            }
        }
    }
}

index_mappings = {
    "properties": {
        "authors": {
            "type": "text",
            "analyzer": "custom_trigram_analyzer",
            "fields": {
                "keyword": {
                    "type": "keyword",
                    "ignore_above": 256
                }
            }
        },
        "num_reviews": {"type": "long"},
        "publish_date": {"type": "date"},
        "publisher": {
            "type": "text",
            "fields": {"keyword": {"type": "keyword", "ignore_above": 256}}
        },
        "summary": {
            "type": "text",
            "fields": {"keyword": {"type": "keyword", "ignore_above": 256}}
        },
        "title": {
            "type": "text",
            "fields": {"keyword": {"type": "keyword", "ignore_above": 256}}
        },
        "title_vector": {
            "type": "dense_vector",
            "dims": 384,
            "index": True,
            "similarity": "cosine",
            "index_options": {"type": "int8_hnsw", "m": 16, "ef_construction": 100}
        }
    }
}

# (Opcional) Nombre del nuevo índice, o puedes usar el mismo si lo vas a borrar primero
NEW_INDEX_NAME = "book_index_trigram" # O usa tu INDEX_NAME original

# 2. (Opcional) Borrar el índice antiguo si vas a reutilizar el nombre y estás en desarrollo
if client.indices.exists(index=NEW_INDEX_NAME):
    print(f"Borrando índice existente: {NEW_INDEX_NAME}")
    client.indices.delete(index=NEW_INDEX_NAME)

# 3. Crear el nuevo índice con la configuración y mapeo
print(f"Creando índice: {NEW_INDEX_NAME}")
client.indices.create(
    index=NEW_INDEX_NAME,
    settings=index_settings,
    mappings=index_mappings
)
print("Índice creado con éxito.")

# 4. Re-poblar tus datos
#    Esto dependerá de cómo cargas tus datos originalmente.
#    Por ejemplo, si tienes una lista de documentos:
#    docs = [
#        {"authors": ["eric matthes"], "title": "Python Crash Course", ...},
#        ...
#    ]
#    for i, doc in enumerate(docs):
#        client.index(index=NEW_INDEX_NAME, id=i, document=doc)
#    print(f"{len(docs)} documentos indexados.")

# Si tienes un índice existente (ej. "book_index") y quieres mover los datos:
print("Iniciando reindexación...")
client.reindex(
     body={
         "source": {"index": "book_index"}, # Tu índice original
         "dest": {"index": NEW_INDEX_NAME}
     },
     request_timeout=300 # Aumenta el timeout si es necesario
 )
print("Reindexación completada.")

In [ ]:
# Asume que 'client' está configurado y NEW_INDEX_NAME es el índice con N-grams
# (y que has re-poblado los datos en este nuevo índice)

# Ejemplo: Buscar "ric"
search_term = "erric"

print(f"\nBuscando '{search_term}' en el campo 'authors' (N-gram):")
response = client.search(
    index=NEW_INDEX_NAME, # Asegúrate de consultar el índice correcto
    query={
        "match": {
            "authors": search_term
        }
    },
    size=5,
    _source=["title", "authors"]
)
# from your_previous_code import pretty_print # si la tienes definida
# pretty_print(response.body)
print(json.dumps(response.body, indent=2, ensure_ascii=False))

# Esto debería encontrar "eric matthes" porque "eric" genera el trigrama "ric",
# y el término de búsqueda "ric" (al ser de longitud 3) se convierte en el token "ric".